In [29]:
import joblib
from pygments.lexers import JavaLexer
from pygments import lex
from pygments.token import Token
import sys
import os

In [30]:
def extract_tokens(code):
    """
    Extracts all tokens from the given Java code using Pygments.
    
    Args:
        code (str): Java code as a string.
        
    Returns:
        str: All tokens extracted from the code, joined by spaces.
    """
    lexer = JavaLexer()
    tokens = []
    for ttype, value in lex(code, lexer):
        # Excluimos solo espacios en blanco para reducir ruido
        if not str(ttype).startswith('Token.Text.Whitespace'):
            val = value.strip()
            if val:
                # IMPORTANTE: Usar exactamente el mismo formato que en el entrenamiento
                # En el entrenamiento usabas f"{ttype}:{val}"
                tokens.append(f"{ttype}:{val}")
    return " ".join(tokens)

In [31]:
def predict_similarity(file1, file2, model_path='rf_model.pkl', vectorizer_path='rf_model_vectorizer.pkl'):
    """
    Predice la similitud entre dos archivos de código Java.

    Args:
        file1 (str): Ruta al primer archivo Java.
        file2 (str): Ruta al segundo archivo Java.
        model_path (str): Ruta al modelo entrenado.
        vectorizer_path (str): Ruta al vectorizador entrenado.

    Returns:
        tuple: (score, is_similar)
    """
    try:
        # Cargar modelo y vectorizador
        model = joblib.load(model_path)
        vectorizer = joblib.load(vectorizer_path)
        
        # Leer archivos con manejo de errores
        try:
            with open(file1, 'r', encoding='utf-8') as f:
                code1 = f.read()
        except UnicodeDecodeError:
            # Intentar con otra codificación si UTF-8 falla
            with open(file1, 'r', encoding='latin-1') as f:
                code1 = f.read()
        
        try:
            with open(file2, 'r', encoding='utf-8') as f:
                code2 = f.read()
        except UnicodeDecodeError:
            with open(file2, 'r', encoding='latin-1') as f:
                code2 = f.read()
        
        # Regla especial para archivos vacíos o muy pequeños
        if len(code1.strip()) < 10 or len(code2.strip()) < 10:
            print("Al menos uno de los archivos está vacío o es muy pequeño.")
            return 0.0, False
        
        # Tokenizar el código completo
        t1 = extract_tokens(code1)
        t2 = extract_tokens(code2)
        token_pair = f"{t1} {t2}"
        
        # Vectorizar usando el mismo vectorizador del entrenamiento
        X = vectorizer.transform([token_pair])
        
        # Para modelos como MLP que no tienen transform, convertimos a array
        X_features = X.toarray()
        
        # Predecir
        if hasattr(model, 'predict_proba'):
            # Para modelos que pueden dar probabilidades
            proba = model.predict_proba(X_features)[0]
            similarity_score = proba[1] if len(proba) > 1 else proba[0]
        else:
            # Para modelos que solo dan la clase
            prediction = model.predict(X_features)[0]
            similarity_score = float(prediction)
        
        # Umbral personalizable
        threshold = 0.50
        is_similar = similarity_score >= threshold
        
        return similarity_score, is_similar
    
    except Exception as e:
        print(f"Error en la predicción: {str(e)}")
        return None, False

In [32]:
   
file1 = "original.java"
file2 = "plagiarized.java"
    
model_path = "mlp_model.pkl"
vectorizer_path = "mlp_model_vectorizer.pkl"
    
# Verificar que los archivos existen
for f in [file1, file2, model_path, vectorizer_path]:
    if not os.path.exists(f):
        print(f"Error: El archivo {f} no existe.")
        sys.exit(1)
    
score, is_similar = predict_similarity(file1, file2, model_path, vectorizer_path)
    
if score is not None:
        print("\n=== RESULTADOS DE LA DETECCIÓN DE PLAGIO ===")
        print(f"Archivo 1: {file1}")
        print(f"Archivo 2: {file2}")
        print(f"Puntaje de similitud: {score:.4f} (0-1)")
        print(f"Plagio detectado: {'SÍ' if is_similar else 'NO'}")
        print("==========================================\n")
        
        # Información adicional para debug
        print("INFORMACIÓN ADICIONAL:")
        print(f"- Tamaño del archivo 1: {os.path.getsize(file1)} bytes")
        print(f"- Tamaño del archivo 2: {os.path.getsize(file2)} bytes")
        print(f"- Modelo utilizado: {os.path.basename(model_path)}")
        
        # Salida con formato para procesamiento automático (si se necesita)
        print(f"\nRESULTADO_JSON: {{'score': {score}, 'is_plagiarism': {is_similar}}}")


=== RESULTADOS DE LA DETECCIÓN DE PLAGIO ===
Archivo 1: original.java
Archivo 2: plagiarized.java
Puntaje de similitud: 0.9999 (0-1)
Plagio detectado: SÍ

INFORMACIÓN ADICIONAL:
- Tamaño del archivo 1: 128 bytes
- Tamaño del archivo 2: 128 bytes
- Modelo utilizado: mlp_model.pkl

RESULTADO_JSON: {'score': 0.9999446264537352, 'is_plagiarism': True}
